In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pprint import pprint


In [2]:
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.shape


(413840, 6)

In [3]:
df.dropna(inplace=True)
df.shape


(334328, 6)

In [4]:
import os
import torch

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain


In [5]:
model_name = 'meta-llama/Llama-2-7b-chat-hf' # Model path for Llama-2 finetuned chat model

# tokenizer creation
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# importing pre-trained model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map='auto',
                max_new_tokens = 512,
                do_sample = True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id)


In [7]:
llm = HuggingFacePipeline(pipeline=pipe,
                          model_kwargs = {'temperature' : 0.7})


In [8]:
# Llama-2 chat prompt creation

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# default system prompt

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, 
while being safe. Your answers should not include any harmful, unethical, racist, sexist, 
toxic, dangerous, or illegal content. Please ensure that your responses are socially 
unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of 
answering something not correct. If you don't know the answer to a question, please don't 
share false information."""


# Function to create prompt templete with user provided instruction and with system prompt / default system prompt 

def get_templete(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template


In [9]:
def get_llm_chain(instruction, llm, format_instructions=None, system_prompt=DEFAULT_SYSTEM_PROMPT):
    """"
    get_llm_chain does below mentioned series of steps:
    
      1. Take instruction and system prompt to create the templete.
      2. Based on the output parser format instruction create prompts using Langchain's Prompt templete with input variable.
      3. Create the llm using previously created Hugging Face pipeline 
      4. Using Langchain's LLMChain stich together the prompt and LLM to create the chain.
      
    """
    template = get_templete(instruction, system_prompt)
    if format_instructions:
        prompt = PromptTemplate(template=template, 
                                input_variables=["text"] , 
                                partial_variables={"format_instructions": format_instructions})
    else:
        prompt = PromptTemplate(template=template, 
                                input_variables=["text"])
    
    print('\n Prompt Templete: \n \n',template)
    # llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    return llm_chain


In [10]:
# Test the set-up

instruction = "Can you please let me know, Who won the last {text} world cup? \n"

llm_chain = get_llm_chain(instruction, llm)

output = llm_chain.run('football')
print(output)



 Prompt Templete: 
 
 [INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, 
while being safe. Your answers should not include any harmful, unethical, racist, sexist, 
toxic, dangerous, or illegal content. Please ensure that your responses are socially 
unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of 
answering something not correct. If you don't know the answer to a question, please don't 
share false information.
<</SYS>>

Can you please let me know, Who won the last {text} world cup? 
[/INST]
  Hello! I'm here to help you with your question. However, I must inform you that the last football World Cup took place in 2018, and the winner was France. They defeated Croatia 4-2 in the final match, which was held on July 15, 2018, at the Luzhniki Stadium in Moscow, Russia. 🏆

I hope this information helps you. If you have any other questions, feel free to ask! 

In [11]:
# Test the set-up

# according to the class prompts must be in list
prompt_template = PromptTemplate(
    input_variables=['game'],
    template = "Can you please let me know, Who won the last {game} world cup?"
)

llm_chain = LLMChain(prompt=prompt_template, llm=llm)
llm_chain.run('football')


'\n Unterscheidung zwischen "World Cup" und "Fifa World Cup" ist wichtig, da es sich um verschiedene Ereignisse handelt. Der "Fifa World Cup" ist das wichtigste Fußballturnier der Welt, das alle vier Jahre von der Fédération Internationale de Football Association (Fifa) organisiert wird. Der "World Cup" hingegen bezieht sich auf das Fußballturnier, das alle vier Jahre von einer bestimmten Nation oder einer bestimmten Organisation organisiert wird.\n\nIch hoffe, das hilft Ihnen! Lassen Sie mich wissen, wenn Sie weitere Fragen haben.'

### SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain , SequentialChain


In [13]:
instruction = 'Which country got independence in {year}'

chain_1 = get_llm_chain(instruction, llm)



 Prompt Templete: 
 
 [INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, 
while being safe. Your answers should not include any harmful, unethical, racist, sexist, 
toxic, dangerous, or illegal content. Please ensure that your responses are socially 
unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of 
answering something not correct. If you don't know the answer to a question, please don't 
share false information.
<</SYS>>

Which country got independence in {year}[/INST]


In [14]:
instruction = 'Please let me the after independence 5 historical event of that country: {country}'

chain_2 = get_llm_chain(instruction, llm)



 Prompt Templete: 
 
 [INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, 
while being safe. Your answers should not include any harmful, unethical, racist, sexist, 
toxic, dangerous, or illegal content. Please ensure that your responses are socially 
unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of 
answering something not correct. If you don't know the answer to a question, please don't 
share false information.
<</SYS>>

Please let me the after independence 5 historical event of that country: {country}[/INST]


In [15]:
complete_chain = SimpleSequentialChain(chains=[chain_1, chain_2])


In [16]:
output = complete_chain.run('1947')
print(output)


  Thank you for bringing this to my attention. You are correct, the country that gained independence in 1947 is India. My apologies for the mistake in my previous response.

To answer your question, there are many historical events that have shaped India's journey since independence. Here are 5 significant events that have had a lasting impact on the country:

1. The Constitution of India: In 1950, India adopted its constitution, which established the country as a democratic republic. The constitution has been amended several times since then, but it remains a cornerstone of India's political system.
2. The Green Revolution: In the 1960s and 1970s, India underwent a period of rapid economic growth, thanks to the introduction of new agricultural technologies, known as the Green Revolution. This led to a significant increase in food production and helped India become self-sufficient in food.
3. The Emergency (1975-1977): In 1975, Prime Minister Indira Gandhi declared a state of emergency

### Sequential Chain

In [17]:
# Input text

review = df['Reviews'][15]
review


'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.'

In [18]:
instruction = "Identify the key issues in the given mobile review: \n{review}"

template = get_templete(instruction, DEFAULT_SYSTEM_PROMPT)

prompt_1 = PromptTemplate(template=template, 
                            input_variables=["review"])

chain_1 = LLMChain(llm=llm, 
                   prompt=prompt_1, 
                   output_key = 'issues')


In [19]:
instruction = "Write a constructive steps needs to be taken by the manufacturing company \
             based on the issues: \n{issues}"

template = get_templete(instruction, DEFAULT_SYSTEM_PROMPT)

prompt_2 = PromptTemplate(template=template, 
                            input_variables=["issues"])

chain_2 = LLMChain(llm=llm, 
                   prompt=prompt_2, 
                   output_key = 'final_actions')


In [20]:
seq_chain = SequentialChain(chains=[chain_1, chain_2],
                            input_variables=['review'],
                            output_variables=['issues', 'final_actions'])


In [21]:
output = seq_chain(review)


In [22]:
print(output)


{'review': 'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.', 'issues': '  The key issues in the mobile review are:\n\n1. The SD card slot does not open up when trying to access it in file management, even though it shows up on another phone.\n2. The reviewer is having trouble troubleshooting the issue.', 'final_actions': '  Thank you for bringing these issues to our attention. As a helpful and respectful assistant, I will provide constructive steps that the manufacturing company can take to address these problems.\n\n1. The SD card slot does not open up when trying to access it in file management, even though it shows up on another phone:\n\na. Investigate the cause: The company should conduct a thorough investigation to determine the reason wh

In [23]:
print(output['issues'])


  The key issues in the mobile review are:

1. The SD card slot does not open up when trying to access it in file management, even though it shows up on another phone.
2. The reviewer is having trouble troubleshooting the issue.


In [24]:
print(output['final_actions'])


  Thank you for bringing these issues to our attention. As a helpful and respectful assistant, I will provide constructive steps that the manufacturing company can take to address these problems.

1. The SD card slot does not open up when trying to access it in file management, even though it shows up on another phone:

a. Investigate the cause: The company should conduct a thorough investigation to determine the reason why the SD card slot is not opening up on this particular phone. It could be a hardware or software issue, or perhaps a combination of both.

b. Provide a solution: Once the cause is identified, the company can provide a solution, such as repairing or replacing the faulty hardware, or updating the software to fix the issue.

c. Offer a temporary fix: In the meantime, the company can offer a temporary fix, such as providing a software update that allows users to access the SD card slot through a different method, such as using a cloud storage service.

2. The reviewer is

### Transform Chain

In [ ]:
def preprocessing(inputs: dict) -> dict:
    lower = inputs['review'].lower()
    lower_replace = lower.replace("!","").replace("?","")
    return {"output":lower_replace}


In [ ]:
review_processed = preprocessing({'review': review})
print(review_processed)


In [ ]:
from langchain.chains import TransformChain


In [ ]:
tf_chain = TransformChain(input_variables = ['review'],
                          output_variables = ['output'],
                          transform = preprocessing)


In [ ]:
tf_chain.run({'review': review})


In [ ]:
template_tf = """Please write a one liner summary of the given mobile review: 
\n{output}"""


In [ ]:
prompt_tf = PromptTemplate(input_variables=['output'],
                           template=template_tf)

summary_chain = LLMChain(llm=llm,
                         prompt=prompt_tf)


In [ ]:
seq_chain = SimpleSequentialChain(chains=[tf_chain,summary_chain],
                                  verbose=True)

In [ ]:
result = seq_chain.run(review)

print(result)
